In [ ]:
!pip install pandas numpy spacy scikit-learn tensorflow datasets spacy_langdetect
!pip install lckr_jupyterlab_variableinspector
!pip install spacy_fastlang

In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import spacy
from spacy_langdetect import LanguageDetector
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split


/home/sahil/spot/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

df = (pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv"))
df = df.dropna()

In [3]:
#get train and test datasets
predict = df['popularity']
df = df.drop(columns=['popularity'])
#ranged stratify split for 0 to 100
y_binned = pd.cut(predict, bins=10, labels=False)
df_train, df_test, df_train_pred, df_test_pred = train_test_split(df, predict, test_size=0.2, random_state=0, stratify=y_binned)


In [4]:
#drop useless columns
df_train.drop(columns=['Unnamed: 0', 'track_id'], inplace=True)
df_test.drop(columns=['Unnamed: 0', 'track_id'], inplace=True)

In [5]:
from sklearn.compose import ColumnTransformer
#TODO FIX THIS IF BROKEN
transformer = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['explicit', 'mode',  'track_genre', 'key', 'time_signature']),
        ('minmax_scale', MinMaxScaler(), ['duration_ms', 'loudness', 'tempo']),
    ], remainder='passthrough') 

#fitting on only train prevents data leakage into test dataset 
transformer.fit(df_train)
df_train = pd.DataFrame(transformer.transform(df_train), columns=transformer.get_feature_names_out())
df_test = pd.DataFrame(transformer.transform(df_test), columns=transformer.get_feature_names_out())


  

In [ ]:
import spacy
from spacy_langdetect import LanguageDetector
import tqdm
import torch
#implement memory limit here
if torch.cuda.is_available():
    spacy.require_gpu()


nlp = spacy.load("en")
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)

def input_batch(df, batch_size):
    results2=[]
    texts = df['remainder_album_name'].astype(str).tolist()

    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        results = []
        for doc in nlp.pipe(batch, batch_size=batch_size, n_process=1):
            results.append(doc._.language['language'])
        results2.extend(results)
    return results2
